In [1]:
import os
import gc
import glob
import datetime
import snappy
from snappy import ProductIO
from collections import OrderedDict
from sentinelsat import SentinelAPI, geojson_to_wkt, read_geojson

In [2]:
input_dir = r'C:\Users\csoti\Desktop\notebooks\SAR\projects\test\1_raw_data'
date_1 = '23Jun2021'
date_2 = '07Jul2021'
query_style = 'footprint'
footprint = r'C:\Users\csoti\Desktop\CyprusFire\input\BurnedArea.geojson'
lat=24.84 
lon=90.43

In [6]:
class sentinel1_download_preprocess():
    def __init__(self, input_dir, date_start, date_end, query_style, footprint, lat=24.84, lon=90.43, download=False):
        self.input_dir = input_dir
        self.date_start = datetime.datetime.strptime(date_1, "%d%b%Y")
        self.date_end = datetime.datetime.strptime(date_2, "%d%b%Y")
        self.query_style = query_style
        self.footprint = geojson_to_wkt(read_geojson(footprint))
        self.lat = lat
        self.lon = lon
        self.download = download

        # configurations
        self.api = SentinelAPI('christiansot', 'belit61218', 'https://scihub.copernicus.eu/dhus')
        self.platform = 'S1B' # 'S1A', None
        self.producttype = 'GRD'  # SLC, GRD, OCN
        self.orbitdirection = 'DESCENDING'  # ASCENDING, DESCENDING
        self.polarisationmode = 'VV VH' # 'VV' 'VH' 'HH' 'HV'
        self.sensoroperationalmode = 'IW'  # SM, IW, EW, WV
        
        
    def sentinel1_download(self):
        global download_candidate
        if self.query_style == 'coordinate':
            download_candidate = self.api.query('POINT({0} {1})'.format(self.lon, self.lat),
                                                date=(self.date_start, self.date_end),
                                                producttype=self.producttype,
                                                orbitdirection=self.orbitdirection,
                                                sensoroperationalmode=self.sensoroperationalmode)
        elif self.query_style == 'footprint':
            download_candidate = self.api.query(self.footprint,
                                                date=(self.date_start, self.date_end),
                                                producttype=self.producttype,
                                                orbitdirection=self.orbitdirection,
                                                polarisationmode=self.polarisationmode,
                                                sensoroperationalmode=self.sensoroperationalmode)
        else:
            print("Define query attribute")

        downls_list = []
        platform = self.platform
        download_products = OrderedDict()
        
        title_found_sum = 0
        for key, value in download_candidate.items():
            for k, v in value.items():
                
                if platform!= None:
                    if k == 'title' and v.split('_')[0]== platform:
                        title_info = v
                        title_found_sum += 1
                        download_products[key] = value
                        downls_list.append(os.path.join(input_dir, title_info + '.zip'))
                        for prodkey, size in value.items():
                            if prodkey == 'size' :
                                print("title: " + title_info + " | " + size)
                                
                if platform== None:
                    if k == 'title':
                        title_info = v
                        title_found_sum += 1
                        downls_list.append(os.path.join(rawdir, title_info + '.zip'))
                        download_products[key] = value
                        for prodkey, size in value.items():
                            if prodkey == 'size' :
                                print("title: " + title_info + " | " + size)
                    
                    
        print("Total found " + str(title_found_sum) +
              " title of " + str(self.api.get_products_size(download_products)) + " GB")

        os.chdir(self.input_dir)
        if self.download:
            if glob.glob(input_dir + "*.zip") not in [value for value in download_candidate.items()]:
                self.api.download_all(download_products)
                print("Nothing to download")
        else:
            print("Escaping download")

In [8]:
sar = sentinel1_download_preprocess(input_dir, date_1, date_2, query_style, footprint, lat, lon, True) 
sar.sentinel1_download()

title: S1B_IW_GRDH_1SDV_20210706T035057_20210706T035122_027668_034D58_6387 | 1.6 GB
title: S1B_IW_GRDH_1SDV_20210701T034259_20210701T034324_027595_034B3C_0452 | 1.61 GB
title: S1B_IW_GRDH_1SDV_20210701T034234_20210701T034259_027595_034B3C_1051 | 1.61 GB
title: S1B_IW_GRDH_1SDV_20210624T035056_20210624T035121_027493_034842_F540 | 1.6 GB
Total found 4 title of 6.42 GB


Downloading:   3%|█▉                                                                | 23.1M/790M [00:43<23:09, 552kB/s]
Downloading:   3%|██                                                                | 25.2M/790M [00:46<21:24, 595kB/s]
Downloading:   0%|                                                                  | 1.05M/774M [00:02<35:13, 366kB/s]
Downloading:   3%|██▏                                                               | 26.2M/790M [00:47<20:27, 622kB/s]
Downloading:   3%|██▎                                                               | 27.3M/790M [00:49<21:21, 595kB/s]
Downloading:   1%|▎                                                                 | 4.19M/774M [00:05<15:16, 840kB/s]
Downloading:   4%|██▎                                                               | 28.3M/790M [00:51<20:02, 633kB/s]
Downloading:   4%|██▍                                                               | 29.4M/790M [00:52<19:38, 645kB/s]
Downloading:   4%|██▌                   

Downloading:   9%|█████▊                                                            | 70.3M/790M [02:14<20:24, 588kB/s]
Downloading:   9%|█████▉                                                            | 71.3M/790M [02:15<19:01, 629kB/s]
Downloading:   9%|█████▉                                                            | 70.3M/774M [01:32<12:59, 903kB/s]
Downloading:   9%|██████                                                            | 72.4M/790M [02:18<21:29, 556kB/s]
Downloading:   9%|██████▏                                                           | 72.4M/774M [01:34<13:10, 888kB/s]
Downloading:   9%|██████▎                                                           | 73.4M/774M [01:35<12:12, 957kB/s]
Downloading:   9%|██████▏                                                           | 73.4M/790M [02:21<24:18, 491kB/s]
Downloading:  10%|██████▎                                                          | 75.5M/774M [01:37<11:31, 1.01MB/s]
Downloading:   9%|██████▏               

Downloading:  14%|█████████▌                                                         | 113M/790M [03:36<21:18, 529kB/s]
Downloading:  18%|███████████▉                                                       | 138M/774M [02:53<17:14, 614kB/s]
Downloading:  14%|█████████▋                                                         | 114M/790M [03:38<22:40, 497kB/s]
Downloading:  18%|████████████▏                                                      | 141M/774M [02:55<12:54, 818kB/s]
Downloading:  15%|█████████▊                                                         | 115M/790M [03:40<21:15, 529kB/s]
Downloading:  18%|████████████▎                                                      | 143M/774M [02:56<11:11, 940kB/s]
Downloading:  15%|█████████▊                                                         | 116M/790M [03:42<21:46, 515kB/s]
Downloading:  19%|████████████▎                                                     | 145M/774M [02:58<10:00, 1.05MB/s]
Downloading:  19%|████████████▌         

Downloading:  27%|█████████████████▌                                                | 207M/774M [03:53<07:58, 1.19MB/s]
Downloading:  27%|█████████████████▋                                                | 208M/774M [03:54<07:39, 1.23MB/s]
Downloading:  27%|█████████████████▊                                                | 209M/774M [03:55<08:32, 1.10MB/s]
Downloading:  27%|█████████████████▉                                                | 210M/774M [03:56<08:33, 1.10MB/s]
Downloading:  27%|█████████████████▉                                                | 211M/774M [03:57<08:41, 1.08MB/s]
Downloading:  27%|██████████████████                                                | 212M/774M [03:58<08:22, 1.12MB/s]
Downloading:  27%|██████████████████▏                                               | 213M/774M [03:59<08:16, 1.13MB/s]
Downloading:  28%|██████████████████▌                                                | 214M/774M [04:00<09:31, 980kB/s]
Downloading:  28%|██████████████████▎   

...

Downloading:  37%|████████████████████████▌                                          | 282M/769M [09:26<13:32, 599kB/s]
Downloading:  37%|████████████████████████▊                                          | 284M/769M [09:29<12:16, 658kB/s]
Downloading:  37%|████████████████████████▊                                          | 285M/769M [09:31<11:08, 724kB/s]
Downloading:  37%|█████████████████████████                                          | 287M/769M [09:33<11:19, 709kB/s]
Downloading:  38%|█████████████████████████▏                                         | 288M/769M [09:36<14:30, 552kB/s]
Downloading:  38%|█████████████████████████▏                                         | 289M/769M [09:38<13:15, 603kB/s]
Downloading:  38%|█████████████████████████▎                                         | 290M/769M [09:39<13:21, 597kB/s]
Downloading:  38%|█████████████████████████▍                                         | 292M/769M [09:42<14:22, 553kB/s]
Downloading:  38%|██████████████████████

Downloading:  46%|██████████████████████████████▉                                    | 354M/769M [11:42<13:18, 519kB/s]
Downloading:  46%|███████████████████████████████                                    | 357M/769M [11:45<11:29, 598kB/s]
Downloading:  47%|███████████████████████████████▏                                   | 358M/769M [11:47<11:16, 608kB/s]
Downloading:  47%|███████████████████████████████▎                                   | 360M/769M [11:50<10:48, 631kB/s]
Downloading:  47%|███████████████████████████████▍                                   | 361M/769M [11:52<12:15, 555kB/s]
Downloading:  47%|███████████████████████████████▌                                   | 363M/769M [11:56<11:35, 584kB/s]
Downloading:  47%|███████████████████████████████▋                                   | 364M/769M [11:57<10:51, 622kB/s]
Downloading:  48%|███████████████████████████████▉                                   | 366M/769M [12:00<10:12, 658kB/s]
Downloading:  48%|██████████████████████

MD5 checksumming:  18%|███████████                                                  | 144M/795M [00:03<00:07, 86.1MB/s]
MD5 checksumming:  19%|███████████▊                                                 | 155M/795M [00:03<00:07, 91.4MB/s]
MD5 checksumming:  21%|████████████▌                                                | 164M/795M [00:03<00:07, 89.8MB/s]
MD5 checksumming:  22%|█████████████▎                                               | 173M/795M [00:03<00:07, 85.6MB/s]
MD5 checksumming:  23%|██████████████                                               | 183M/795M [00:03<00:06, 90.8MB/s]
MD5 checksumming:  25%|███████████████▎                                              | 196M/795M [00:03<00:05, 101MB/s]
MD5 checksumming:  26%|████████████████▎                                             | 208M/795M [00:04<00:05, 107MB/s]
MD5 checksumming:  28%|█████████████████▏                                            | 220M/795M [00:04<00:05, 110MB/s]
MD5 checksumming:  29%|█████████████████

Downloading:  52%|███████████████████████████████████                                | 403M/769M [13:06<10:49, 563kB/s]
MD5 checksumming:  87%|█████████████████████████████████████████████████████▏       | 692M/795M [00:11<00:01, 73.5MB/s]
MD5 checksumming:  88%|█████████████████████████████████████████████████████▊       | 701M/795M [00:11<00:01, 75.9MB/s]
MD5 checksumming:  89%|██████████████████████████████████████████████████████▎      | 708M/795M [00:11<00:01, 71.4MB/s]
MD5 checksumming:  90%|███████████████████████████████████████████████████████      | 717M/795M [00:11<00:01, 74.4MB/s]
MD5 checksumming:  91%|███████████████████████████████████████████████████████▌     | 724M/795M [00:11<00:00, 74.5MB/s]
MD5 checksumming:  92%|████████████████████████████████████████████████████████▍    | 735M/795M [00:11<00:00, 83.9MB/s]
MD5 checksumming:  94%|█████████████████████████████████████████████████████████    | 743M/795M [00:11<00:00, 83.6MB/s]
MD5 checksumming:  95%|█████████████████

Nothing to download
